In [2]:
!pip install transformers

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F

from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

import random
import os
% matplotlib inline

     |████████████████████████████████| 2.3MB 17.5MB/s 
     |████████████████████████████████| 901kB 47.0MB/s 
     |████████████████████████████████| 3.3MB 37.6MB/s 


In [3]:
df = pd.read_csv("/content/drive/MyDrive/DHBW Vorlesungen/6. Semester/Daten für Projekte/scraped/processed_data.csv")

In [4]:
torch.cuda.empty_cache()

In [5]:
dic = {
    'Mostly True'    : 0.75,
    'False'          : 0.15,
    'True'           : 1.0,
    'Mostly False'   : 0.35, 
    'Legend'         : 0.0,
    'Mixture'        : 0.5,
    'Out'            : None,
    'Scam'           : 0.0,
}

In [6]:
type(dic["True"])

float

In [7]:
df.replace({"label": dic}, inplace=True)

In [8]:
df.dropna(subset=['label'],inplace=True)

In [9]:
cuda_available = torch.cuda.is_available()
cuda_available

True

In [10]:
df.drop(columns=['author', 'claim', 'title'],
             inplace=True)

In [11]:
cols = ['text', 'label']
df = df[cols]

In [12]:
"""
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])
"""

'\ntrain, validate, test =               np.split(df.sample(frac=1, random_state=42), \n                       [int(.6*len(df)), int(.8*len(df))])\n'

In [13]:
print("Distribution of data based on labels: ",df.label.value_counts())

Distribution of data based on labels:  0.15    12744
0.50     7319
0.35     6333
0.75     5226
1.00     4555
0.00     2846
Name: label, dtype: int64


In [14]:
max = 0
for text in df.text:
  text = text.split(' ')
  if len(text) > max:
    max = len(text)
max

6614

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [16]:
sentences = df.text.values

In [17]:
input_ids = [tokenizer.encode(sent[:512], add_special_tokens=True,max_length=512, pad_to_max_length=True) if len(sent)>512 else tokenizer.encode(sent, add_special_tokens=True,max_length=512, pad_to_max_length=True) for sent in sentences]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [18]:
labels = df.label.values

In [19]:
print("Actual sentence before tokenization: ",sentences[2])
print("Encoded Input from dataset: ",input_ids[2])

Actual sentence before tokenization:  Theres a big, slow-news-weekend story over at Politico today over allegations made by certain Democrats and liberals that the prolific polling firm Rasmussen Reports is biased toward conservative and Republican causes. I have to catch a flight (weather permitting) in a couple of hours, so I dont have quite as much time to weigh in on this as Id like, but let me present the Cliffs Notes version. The first thing to note is that there are a lot of different ways in which a polling firm might be biased. Rasmussen is most frequently accused of bias because their results are thought to lean toward Republican candidates. Just to pick a random example, for instance, Rasmussen has embattled Democrat Blanche Lincoln down by margins ranging between 4 and 7 points against three potential Republican opponents in her 2010 Senate race, whereas two other polling firms (neither of which, incidentally, are themselves free of partisan ties) have Lincoln ahead against

In [20]:
attention_masks = []
## Create a mask of 1 for all input tokens and 0 for all padding tokens
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
print(len(attention_masks))

39023


In [21]:
from sklearn.model_selection import train_test_split
train_inputs, val_inputs_help, train_labels, val_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.4)
train_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.4)

val_inputs, test_inputs, val_labels, test_labels = train_test_split(val_inputs_help, val_labels, random_state=42, test_size=0.5)
val_masks, test_masks, _, _ = train_test_split(val_masks, val_inputs_help, random_state=42, test_size=0.5)

In [22]:
# convert all our data into torch tensors, required data type for our model
train_inputs = torch.tensor(train_inputs)
val_inputs = torch.tensor(val_inputs)
test_inputs = torch.tensor(test_inputs)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
test_masks = torch.tensor(test_masks)

In [23]:
batch_size = 5

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs,train_masks,train_labels.float())
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

val_data = TensorDataset(val_inputs,val_masks,val_labels.float())
val_sampler = RandomSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels.float())
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [25]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [26]:
# Parameters:
lr = 2e-3
adam_epsilon = 1e-8

# Number of training epochs (authors recommend between 2 and 4)
epochs = 1

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon,correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

In [27]:
torch.cuda.memory_summary(device=None, abbreviated=False)
#torch.cuda.empty_cache()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  428778 KB |  428778 KB |  428778 KB |       0 B  |\n|       from large pool |  428288 KB |  428288 KB |  428288 KB |       0 B  |\n|       from small pool |     490 KB |     490 KB |     490 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  428778 KB |  428778 KB |  428778 KB |       0 B  |\n|       from large pool |  428288 KB |  428288 KB |

In [28]:
## Store our loss and accuracy for plotting
train_loss_set = []
learning_rate = []

# Gradients gets accumulated by default
model.zero_grad()


# tnrange is a tqdm wrapper around the normal python range
for _ in trange(1,epochs+1,desc='Epoch'):
  print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
  # Calculate total loss for this epoch
  batch_loss = 0

  for step, batch in enumerate(train_dataloader):
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # print(b_input_ids[0].dtype ,"\n", b_input_mask[0].dtype, "\n", b_labels[0].dtype)

    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]

    loss = loss.float()

    # Backward pass
    loss.backward()
    
    # Clip the norm of the gradients to 1.0
    # Gradient clipping is not in AdamW anymore
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    # Update learning rate schedule
    scheduler.step()

    # Clear the previous accumulated gradients
    optimizer.zero_grad()
    
    # Update tracking variables
    batch_loss += loss.item()

  # Calculate the average loss over the training data.
  avg_train_loss = batch_loss / len(train_dataloader)

  #store the current learning rate
  for param_group in optimizer.param_groups:
    print("\n\tCurrent Learning rate: ",param_group['lr'])
    learning_rate.append(param_group['lr'])
    
  train_loss_set.append(avg_train_loss)
  print(F'\n\tAverage Training loss: {avg_train_loss}')
    
  # Validation

  """
  # Umschreiben zu einer Standartabweichung, da Regression keine Klassifikation
  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_accuracy,eval_mcc_accuracy,nb_eval_steps = 0, 0, 0

  # Evaluate data for one epoch
  for batch in val_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits[0].to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()
    tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)
    tmp_eval_mcc_accuracy = matthews_corrcoef(labels_flat, pred_flat)
    
    eval_accuracy += tmp_eval_accuracy
    eval_mcc_accuracy += tmp_eval_mcc_accuracy
    nb_eval_steps += 1

  print(F'\n\tValidation Accuracy: {eval_accuracy/nb_eval_steps}')
  print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy/nb_eval_steps}')
  """

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

<====================== Epoch 1 ======================>


Epoch: 100%|██████████| 1/1 [41:19<00:00, 2479.01s/it]


	Current Learning rate:  0.0

	Average Training loss: 0.2619276971441596
